<a href="https://colab.research.google.com/github/Waflon/Transparencia-Senadores-Scrap/blob/main/Asistencia_Senadores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Guardar en Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#Get CSV

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
pd.set_option('display.max_rows', 50)

def get_Table(id_periodo: int, flag: bool) -> pd:
  url_lanzador = f'https://www.senado.cl/appsenado/lanzador.php?mo=sesionessala&ac=asistenciaSenadores&camara=S&legiini=361&legiid={id_periodo}'

  with requests.Session() as s:  # iniciar en lanzador y luego en index
    r = s.get(url_lanzador)  
    url_index = f'https://www.senado.cl/appsenado/index.php?mo=sesionessala&ac=asistenciaSenadores&camara=S&legiini=361&legiid={id_periodo}'
    r = s.get(url_index)
    soup = BeautifulSoup(r.text, 'lxml')  # guardar parseado

  #Arreglos de la tabla con Beautiful soup
  table = soup.find_all('table')
  df = pd.read_html(str(table))[1].fillna(0)
  df = df[df['Senador'].apply(lambda x: isinstance(x, (str, bytes)))]
  df.reset_index(inplace=True)
  df.drop('index', axis=1, inplace=True)
  if (flag):
    lista_fechas = []
    for fecha in soup.find_all('option'):
      lista_fechas.append(int(fecha['value']))
    return lista_fechas

  return soup, df
  

Obtener los rangos de fecha disponibles para navegar por los GET

get and save CSV

In [ ]:
lista_fechas = get_Table(501, True) #id inicial

def generar_CSV(id_fecha):
  soup, df = get_Table(id_fecha, False)
  fecha = soup.find('option', {'selected': True})
  rango_fecha = fecha.text[11:15] + '-' + fecha.text[24:-1]
  df.to_csv(f'Asistencia_Sesiones_Senadores_{rango_fecha}.csv')
  !sudo cp 'Asistencia_Sesiones_Senadores_{rango_fecha}.csv' '/content/drive/My Drive/Colab Notebooks/Senadores/Asistencia_Sesiones_Senadores_{rango_fecha}.csv'
  

In [ ]:
for fecha in lista_fechas:
  generar_CSV(fecha)

---
# Plotting
---

In [ ]:
from IPython.core.excolors import exception_colors
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from PIL import Image

def get_Table(id_periodo : int) -> BeautifulSoup:
  url_lanzador = f'https://www.senado.cl/appsenado/lanzador.php?mo=sesionessala&ac=asistenciaSenadores&camara=S&legiini=361&legiid={id_periodo}'

  with requests.Session() as s:  # iniciar en lanzador y luego en index
    r = s.get(url_lanzador)  
    url_index = f'https://www.senado.cl/appsenado/index.php?mo=sesionessala&ac=asistenciaSenadores&camara=S&legiini=361&legiid={id_periodo}'
    r = s.get(url_index)
    soup = BeautifulSoup(r.text, 'lxml')  # guardar parseado
  return soup

def get_DataFrame(soup):
  #Arreglos de la tabla con Beautiful soup
  table = soup.find_all('table')
  df = pd.read_html(str(table))[1].fillna(0)
  df = df[df['Senador'].apply(lambda x: isinstance(x, (str, bytes)))]
  df.reset_index(inplace=True)
  df.drop('index', axis=1, inplace=True)
  return df

def get_Fechas(soup):
  lista_fechas = []
  for fecha in soup.find_all('option'):
    lista_fechas.append(int(fecha['value']))
  return lista_fechas

def get_Sesiones_totales(soup) -> pd:
  try:
    sesiones_totales = soup.find("h2").text[-3:]
  except:
    sesiones_totales = 130
  return sesiones_totales

def dibujar(df, n_sesiones):
  figure(figsize=(15, 10), dpi=80)
  largo = len(df[:])

  sesiones=np.empty(largo)
  sesiones.fill(n_sesiones)

  sesiones_asistidas = df['Asistencia'].to_numpy(dtype="int")
  inasistencias_justificadas = df['Inasistencias justificadas'].to_numpy(dtype="int")
  df['promedio'] = df['Asistencia'].astype(int) + df['Inasistencias justificadas'].astype(int)
  df['promedio'] = df['promedio'].astype(int).div(int(n_sesiones)).mul(100).round(2)

  df['Senador'] = df['Senador'].astype(str) + ' ' + df['promedio'].astype(str) + '%'
  senadores = df['Senador'].to_numpy(dtype="str")

  fecha = soup.find('option', {'selected': True})
  rango_fecha = fecha.text[5:15] + fecha.text[15:-1]

  ind = np.arange(largo)    
  width = 0.5      

  p1 = plt.bar(ind, sesiones, width, color='#27d6d5') #plt.plot(x, y, 'r-', alpha=0.7)
  p2 = plt.bar(ind, sesiones_asistidas, width,  bottom=0)
  p3 = plt.bar(ind, inasistencias_justificadas, width,  bottom=sesiones_asistidas)
  
  plt.ylabel(f'Sesiones Totales: {n_sesiones}')
  plt.title(f'Asistencia de Senadores por Periodo {rango_fecha}')
  plt.xticks(ind, (senadores), rotation='vertical')
  plt.yticks(np.arange(0, int(n_sesiones), 10), rotation='vertical')
  plt.legend((p1[0], p2[0], p3[0]), ('Sesiones totales', 'Sesiones Asistidas', "Inasistencias Justificadas"), loc = "center", fontsize=20, ncol = 3)


  plt.tight_layout()
  
  fecha = fecha.text[11:15] + '-' + fecha.text[24:-1]
  plt.savefig(f'Asistencia_Sesiones_Senadores_{fecha}.png')
  
  plt.show()


In [ ]:
def save_img(soup) -> None:
  df = get_DataFrame(soup)
  n_sesiones = get_Sesiones_totales(soup)
  dibujar(df, int(n_sesiones))
  fecha = soup.find('option', {'selected': True})
  fecha = fecha.text[11:15] + '-' + fecha.text[24:-1]
  im = Image.open(f"Asistencia_Sesiones_Senadores_{fecha}.png")
  out = im.transpose(Image.ROTATE_270)
  #out=im.rotate(270, expand=1)
  out.save(f'Asistencia_Sesiones_Senadores_{fecha}_H.png')

In [ ]:
soup = get_Table(501)
fechas = get_Fechas(soup)

In [ ]:
for fecha in fechas:
  soup = get_Table(fecha)
  save_img(soup)